**Upload the 3 CSV Files to Colab**

In [1]:
from google.colab import files
uploaded = files.upload()

Saving bank_marketing.csv to bank_marketing.csv
Saving project_management.csv to project_management.csv
Saving salary_prediction.csv to salary_prediction.csv


In [2]:
import os
print(os.listdir())

['.config', 'project_management.csv', 'bank_marketing.csv', 'salary_prediction.csv', 'sample_data']


**Import required libraries**

In [3]:
import pandas as pd
import numpy as np

**Load the three datasets**

*Replace filenames below with your exact uploaded names.*

In [6]:
project_df = pd.read_csv("project_management.csv")
salary_df  = pd.read_csv("salary_prediction.csv")
bank_df    = pd.read_csv("bank_marketing.csv", sep=';')

print(project_df.head())
print(salary_df.head())
print(bank_df.head())

                Project Name  \
0                 Rhinestone   
1     A Triumph Of Softwares   
2              The Blue Bird   
3  Remembering Our Ancestors   
4                   Skyhawks   

                                 Project Description  \
0  Associations Now Is A Casual Game To Teach You...   
1  Is A Fully Managed Content Marketing Software ...   
2  Most Content Marketers Know The Golden Rule: Y...   
3  Utilize And Utilizes (Verb Form) The Open, Inc...   
4  Is A Solution For Founders Who Want To Win At ...   

                  Project Type  Project Manager Region           Department  \
0            INCOME GENERATION      Yael Wilcox  North           Admin & BI   
1            INCOME GENERATION  Brenda Chandler   West            eCommerce   
2            INCOME GENERATION    Nyasia Hunter  North            Warehouse   
3          PROCESS IMPROVEMENT  Brenda Chandler   East  Sales and Marketing   
4  WORKING CAPITAL IMPROVEMENT  Jaylyn Mckenzie   East            eCommerce

In [11]:
print('Missing values in bank_df:')
print(bank_df.isnull().sum())

Missing values in bank_df:
age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


In [10]:
# Impute numerical columns with the mean
salary_df['Age'] = salary_df['Age'].fillna(salary_df['Age'].mean())
salary_df['Years of Experience'] = salary_df['Years of Experience'].fillna(salary_df['Years of Experience'].mean())
salary_df['Salary'] = salary_df['Salary'].fillna(salary_df['Salary'].mean())

# Impute categorical columns with the mode
salary_df['Gender'] = salary_df['Gender'].fillna(salary_df['Gender'].mode()[0])
salary_df['Education Level'] = salary_df['Education Level'].fillna(salary_df['Education Level'].mode()[0])
salary_df['Job Title'] = salary_df['Job Title'].fillna(salary_df['Job Title'].mode()[0])

print('Missing values in salary_df after imputation:')
print(salary_df.isnull().sum())

Missing values in salary_df after imputation:
Age                    0
Gender                 0
Education Level        0
Job Title              0
Years of Experience    0
Salary                 0
dtype: int64


In [8]:
print('Missing values in salary_df:')
print(salary_df.isnull().sum())

Missing values in salary_df:
Age                    2
Gender                 2
Education Level        2
Job Title              2
Years of Experience    2
Salary                 2
dtype: int64


In [7]:
print('Missing values in project_df:')
print(project_df.isnull().sum())

Missing values in project_df:
Project Name           0
Project Description    0
Project Type           0
Project Manager        0
Region                 0
Department             0
 Project Cost          0
 Project Benefit       0
Complexity             0
Status                 0
Completion%            0
Phase                  0
Year                   0
Month                  0
Start Date             0
End Date               0
dtype: int64


**Clean column names**

*SQL and merging work better with clean names.*

In [12]:
def clean_columns(df):
    df.columns = [c.strip().replace(" ", "_").lower() for c in df.columns]
    return df

project_df = clean_columns(project_df)
salary_df  = clean_columns(salary_df)
bank_df    = clean_columns(bank_df)

print(project_df.columns)

Index(['project_name', 'project_description', 'project_type',
       'project_manager', 'region', 'department', 'project_cost',
       'project_benefit', 'complexity', 'status', 'completion%', 'phase',
       'year', 'month', 'start_date', 'end_date'],
      dtype='object')


**Inspect Structure and Identify Merge Keys**

We need a common key like:

department
project_type
team

Check columns:

In [13]:
print(project_df.columns)
print(salary_df.columns)
print(bank_df.columns)

Index(['project_name', 'project_description', 'project_type',
       'project_manager', 'region', 'department', 'project_cost',
       'project_benefit', 'complexity', 'status', 'completion%', 'phase',
       'year', 'month', 'start_date', 'end_date'],
      dtype='object')
Index(['age', 'gender', 'education_level', 'job_title', 'years_of_experience',
       'salary'],
      dtype='object')
Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


*If datasets DO NOT naturally share a column like department, we will simulate a banking scenario realistically.*

**Create Synthetic Department Mapping**

*We simulate banking departments for demonstration:*

In [14]:
departments = ['retail_banking', 'credit_risk', 'operations', 'marketing', 'it']

project_df['department'] = np.random.choice(departments, len(project_df))
salary_df['department']  = np.random.choice(departments, len(salary_df))
bank_df['department']    = np.random.choice(departments, len(bank_df))

*Now all three datasets have a common merge key: department.*

**Prepare Salary Dataset** (For FTE Cost Modeling)

We need:

salary and
fte_reduction (we will simulate FTE savings per project)

If salary dataset has salary column:

In [15]:
salary_df = salary_df[['department','salary']].copy()

*Now aggregate salary per department.*

In [16]:
salary_summary = salary_df.groupby('department')['salary'].mean().reset_index()
salary_summary.rename(columns={'salary':'avg_salary'}, inplace=True)

print(salary_summary)

       department     avg_salary
0     credit_risk  102085.420076
1              it   96917.808219
2       marketing  101049.382716
3      operations  106589.586656
4  retail_banking   95005.384615


**Prepare Project Dataset**

Check if it already has:

planned_cost
actual_cost

If not, simulate realistic banking project costs:

In [17]:
project_df['planned_cost'] = np.random.randint(500000,3000000,size=len(project_df))
project_df['actual_cost']  = project_df['planned_cost'] * np.random.uniform(0.8,1.2,size=len(project_df))

*Simulate FTE reduction per project:*

In [22]:
project_df['fte_reduction'] = np.random.randint(0,5,size=len(project_df))

**Merge Project + Salary**

In [23]:
merged_df = project_df.merge(
    salary_summary,
    on='department',
    how='left'
)

print(merged_df.head())

                project_name  \
0                 Rhinestone   
1     A Triumph Of Softwares   
2              The Blue Bird   
3  Remembering Our Ancestors   
4                   Skyhawks   

                                 project_description  \
0  Associations Now Is A Casual Game To Teach You...   
1  Is A Fully Managed Content Marketing Software ...   
2  Most Content Marketers Know The Golden Rule: Y...   
3  Utilize And Utilizes (Verb Form) The Open, Inc...   
4  Is A Solution For Founders Who Want To Win At ...   

                  project_type  project_manager region      department  \
0            INCOME GENERATION      Yael Wilcox  North  retail_banking   
1            INCOME GENERATION  Brenda Chandler   West       marketing   
2            INCOME GENERATION    Nyasia Hunter  North              it   
3          PROCESS IMPROVEMENT  Brenda Chandler   East       marketing   
4  WORKING CAPITAL IMPROVEMENT  Jaylyn Mckenzie   East       marketing   

     project_cost project

Now we have:

planned_cost,
actual_cost,
fte_reduction, and
avg_salary

**Create Calculated Columns**

*Cost Variance*

In [24]:
merged_df['cost_variance'] = (
    merged_df['planned_cost'] - merged_df['actual_cost']
)

merged_df['cost_variance_pct'] = (
    merged_df['cost_variance'] / merged_df['planned_cost'] * 100
).round(2)

*FTE Savings Estimation*

Formula:

FTE Savings = Avg Salary × FTE Reduction

In [25]:
merged_df['fte_savings'] = (
    merged_df['avg_salary'] * merged_df['fte_reduction']
).round(2)

*ROI Per Project*

Formula:

ROI = (FTE Savings / Actual Cost) × 100

In [26]:
merged_df['roi_pct'] = (
    merged_df['fte_savings'] / merged_df['actual_cost'] * 100
).round(2)

**Portfolio Prioritization Score**

*Normalize first*

In [27]:
merged_df['roi_norm'] = merged_df['roi_pct'] / merged_df['roi_pct'].max()
merged_df['fte_norm'] = merged_df['fte_savings'] / merged_df['fte_savings'].max()
merged_df['variance_norm'] = merged_df['cost_variance'] / merged_df['cost_variance'].max()

*Weighted Score*

In [29]:
merged_df['portfolio_score'] = (
    0.4 * merged_df['roi_norm'] +
    0.3 * merged_df['fte_norm'] +
    0.3 * merged_df['variance_norm']
).round(3)

merged_df['priority_rank'] = merged_df['portfolio_score'].rank(ascending=False)

**Final Dataset Check**

In [30]:
merged_df.head()
merged_df.describe()

,year,month,planned_cost,actual_cost,fte_reduction,avg_salary,cost_variance,cost_variance_pct,fte_savings,roi_pct,roi_norm,fte_norm,variance_norm,portfolio_score,priority_rank
count,99.000000,99.000000,9.900000e+01,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,2022.747475,7.151515,1.694864e+06,1.678281e+06,1.929293,100569.832643,16583.377916,0.352424,194723.257374,15.651616,0.216183,0.456713,0.036219,0.234303,50.000000
std,1.402210,3.211471,7.853213e+05,7.895317e+05,1.408810,4159.848886,185023.477680,11.366870,143046.277982,15.850496,0.218930,0.335507,0.404104,0.215661,28.722369
min,2021.000000,1.000000,5.460070e+05,5.471361e+05,0.000000,95005.384615,-425717.505529,-19.640000,0.000000,0.000000,0.000000,0.000000,-0.929795,-0.213000,1.000000
25%,2022.000000,4.500000,9.559880e+05,9.419613e+05,1.000000,96917.808219,-126977.259282,-9.210000,95961.595000,4.410000,0.060912,0.225073,-0.277327,0.075500,25.500000
50%,2022.000000,7.000000,1.529798e+06,1.484608e+06,2.000000,101049.382716,15716.999911,0.830000,193835.620000,11.980000,0.165470,0.454631,0.034327,0.231000,50.000000
75%,2024.000000,10.000000,2.512906e+06,2.370323e+06,3.000000,102085.420076,160171.796319,9.585000,313012.510000,21.495000,0.296892,0.734154,0.349826,0.395500,74.500000
max,2025.000000,12.000000,2.996655e+06,3.343083e+06,4.000000,106589.586656,457861.494695,19.070000,426358.350000,72.400000,1.000000,1.000000,1.000000,0.768000,99.000000


**Save CLean Dataset for Power BI & Mode**

In [31]:
merged_df.to_csv('final_project_portfolio.csv', index=False)

from google.colab import files
files.download('final_project_portfolio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Save SQLite (For SQL and Mode Demo)**

In [32]:
import sqlite3

conn = sqlite3.connect('project_portfolio.db')
merged_df.to_sql('project_portfolio', conn, if_exists='replace', index=False)

files.download('project_portfolio.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎯 At This Point You Have:

✔ Raw Kaggle data
✔ Cleaned & standardized columns
✔ Simulated realistic banking structure
✔ Merged salary + project cost data
✔ Cost variance
✔ FTE savings
✔ ROI
✔ Portfolio prioritization ranking
✔ CSV for Power BI
✔ SQLite DB for SQL demonstration